In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=[]

env: CUDA_VISIBLE_DEVICES=[]


### Zebra Finch vocalizations
Source:
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0181992
- https://osf.io/bx76r/
- http://localhost:8186/notebooks/tsainbur/Projects/github_repos/animalvocalizationgenerativenet/notebooks/birdsong/zebra_finch/1.0-segment-song-from-wavs.ipynb

This dataset is larger than the other, but only contains songs, and from fewer individuals

In [2]:
from pathlib2 import Path
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import IPython
import seaborn as sns

In [3]:
from avgn_paper.signalprocessing import spectrogramming as sg

In [4]:
import pdb
from joblib import Parallel, delayed
n_jobs = 10
verbosity = 5

In [5]:
from tqdm.autonotebook import tqdm
from librosa.core.time_frequency import mel_frequencies
from avgn_paper.signalprocessing.spectrogramming_tf import spectrogram_tensorflow
from avgn_paper.utils.audio import load_wav, float32_to_int16, int16_to_float32, write_wav
from avgn_paper.visualization.spectrogram import visualize_spec, plot_spec
from avgn_paper.utils.general import HParams
from avgn_paper.signalprocessing.filtering import butter_bandpass_filter
from datetime import datetime

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### data locations

In [6]:
DSLOC = Path('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/')

In [7]:
MATFILES = list(DSLOC.expanduser().glob('*/*.mat'))
MATFILES = [i for i in MATFILES if i.stem[0]!='.']
len(MATFILES)

8

In [11]:
MATFILES

[PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lr13/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lr77/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lr28/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lny46/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lny4rb/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lr12/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lny64/song.mat'),
 PosixPath('/mnt/cube/Datasets/ZebraFinch/OSF-BX76R/lny42/song.mat')]

In [8]:
import h5py as h5

In [9]:
for mat_file in tqdm(MATFILES):
    indv = mat_file.parent.stem
    # try loading the MAT file
    try:
        with h5.File(mat_file) as f: 
            song = f['song'].value
            rate = f['fs'].value[0]
    except: 
        print(indv +' failed')
        continue
    break

In [10]:
np.shape(song)

(3000, 22923)